<a href="https://colab.research.google.com/github/harshhrawte/NLP/blob/main/Document%20Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gdown PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00


In [ ]:
!pip install pdfplumber gdown
!pip install pymupdf gdown


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 37.5 MB/s eta 0:00:00


In [ ]:

!pip install pymupdf gdown streamlit pandas pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.7 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF
import gdown
import os
import json
import re


def get_file_id(drive_input):
    if "drive.google.com" in drive_input:
        try:
            return drive_input.split("/d/")[1].split("/")[0]
        except IndexError:
            raise ValueError("Invalid Google Drive link format.")
    return drive_input


def download_pdf(file_id, output_path="downloaded.pdf"):
    url = f"https://drive.google.com/uc?id={file_id}"
    print(f"Downloading PDF from: {url}")
    gdown.download(url, output_path, quiet=False)
    return output_path


def extract_text_lines_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    all_lines = []

    print("\nExtracting visible text from pages...\n")
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        lines = [line.strip() for line in text.splitlines() if line.strip()]
        all_lines.extend(lines)

    return all_lines


def split_headers_and_table(lines):
    """
    Identify the table heading and separate lines before and after it.
    """
    headers = []
    table_lines = []

    # Look for the index where the product table starts
    table_start_idx = -1
    for i, line in enumerate(lines):
        if re.search(r"PRODUCT NAME.*ITEM.*SIZE.*COLOUR.*TOTAL", line.upper()):
            table_start_idx = i + 1
            break

    if table_start_idx == -1:
        headers = lines
        return headers, table_lines

    headers = lines[:table_start_idx - 1]
    table_lines = lines[table_start_idx:]
    return headers, table_lines


def structure_table_lines(table_lines):
    """
    Group every 7 lines into a product entry.
    Assumes fixed format: Product Name, Item #, Price, Qty, Size, Color, Total
    """
    products = []
    chunk_size = 7
    for i in range(0, len(table_lines), chunk_size):
        row = table_lines[i:i + chunk_size]
        if len(row) == chunk_size:
            product = {
                "Product Name": row[0],
                "Item #": row[1],
                "Price": row[2],
                "Qty": row[3],
                "Size": row[4],
                "Color": row[5],
                "Total": row[6]
            }
            products.append(product)
    return products


def extract_structured_json(pdf_path):
    lines = extract_text_lines_from_pdf(pdf_path)
    headers, table_lines = split_headers_and_table(lines)
    structured_table = structure_table_lines(table_lines)

    return {
        "Pages": [
            {
                "Page_Number": 1,
                "Headers": headers,
                "List_items": structured_table
            }
        ]
    }


def main():
    print("=== Advanced PDF Extractor + Table Structuring ===")
    drive_input = input("Enter Google Drive file link or file ID: ").strip()

    try:
        file_id = get_file_id(drive_input)
        pdf_path = download_pdf(file_id)

        extracted_data = extract_structured_json(pdf_path)

        with open("extracted_data.json", "w", encoding="utf-8") as f:
            json.dump(extracted_data, f, indent=4, ensure_ascii=False)

        print("\n=== Final Structured JSON Output ===\n")
        print(json.dumps(extracted_data, indent=4, ensure_ascii=False))
        print("\n✅ Data saved to extracted_data.json")

    except Exception as e:
        print(f"\n❌ Error: {e}")
    finally:
        if os.path.exists("downloaded.pdf"):
            os.remove("downloaded.pdf")


if __name__ == "__main__":
    main()


=== Advanced PDF Extractor + Table Structuring ===
Enter Google Drive file link or file ID: https://drive.google.com/file/d/1zeC7WzV6dQUvbrKODZsMbBM0IEka2g7o/view?usp=drive_link 


Downloading...
From: https://drive.google.com/uc?id=1zeC7WzV6dQUvbrKODZsMbBM0IEka2g7o
To: /content/downloaded.pdf
100%|██████████| 229k/229k [00:00<00:00, 79.3MB/s]


Extracting visible text from pages...


=== Final Structured JSON Output ===

{
    "Pages": [
        {
            "Page_Number": 1,
            "Headers": [
                "Name:",
                "PRICE",
                "QTY",
                "Discount:",
                "Taxes:",
                "Shipping:",
                "TOTAL:",
                "Payment Type:",
                "Track #:",
                "Arrival Date:",
                "Method:",
                "Company:",
                "Subtotal:",
                "PRODUCT NAME",
                "ITEM #",
                "SIZE",
                "COLOUR",
                "TOTAL",
                "Address:",
                "Email:",
                "Phone:",
                "DATE:",
                "PURCHASE ORDER #:",
                "CLIENT INFORMATION",
                "ORDER INFORMATION",
                "SHIPPING INFO",
                "PAYMENT INFO",
                "NOTES",
                "22ND SEPTEMBER, 2022"

In [ ]:
import fitz  # PyMuPDF
import gdown
import os
import json


def get_file_id(drive_input):
    if "drive.google.com" in drive_input:
        try:
            return drive_input.split("/d/")[1].split("/")[0]
        except IndexError:
            raise ValueError("Invalid Google Drive link format.")
    return drive_input


def download_pdf(file_id, output_path="downloaded.pdf"):
    url = f"https://drive.google.com/uc?id={file_id}"
    print(f"\n📥 Downloading PDF from: {url}\n")
    gdown.download(url, output_path, quiet=False)
    return output_path


def extract_text_from_fields(pdf_path):
    doc = fitz.open(pdf_path)
    fields = []

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            if block["type"] == 0:
                for line in block["lines"]:
                    for span in line["spans"]:
                        text = span["text"].strip()
                        if text:
                            fields.append(text)
    return fields


def separate_data(texts):
    headers = []
    table = []

    # Detect table start
    try:
        idx = texts.index("PRODUCT NAME ITEM # SIZE COLOUR TOTAL") + 1
    except ValueError:
        idx = -1

    if idx != -1:
        headers = texts[:idx - 1]
        table_data = texts[idx:]

        # Group every 7 items as a table row
        for i in range(0, len(table_data), 7):
            row = table_data[i:i+7]
            if len(row) == 7:
                table.append({
                    "Product Name": row[0],
                    "Item #": row[1],
                    "Price": row[2],
                    "Qty": row[3],
                    "Size": row[4],
                    "Color": row[5],
                    "Total": row[6],
                })
    else:
        headers = texts  # fallback: everything is header

    return headers, table


def extract_to_json(pdf_path):
    texts = extract_text_from_fields(pdf_path)
    headers, table = separate_data(texts)

    return {
        "Pages": [
            {
                "Page_Number": 1,
                "Headers": headers,
                "List_items": table
            }
        ]
    }


def main():
    print("=== NLP-Ready PDF Extractor ===")
    drive_input = input("📎 Enter Google Drive file link or file ID: ").strip()

    try:
        file_id = get_file_id(drive_input)
        pdf_path = download_pdf(file_id)

        extracted_data = extract_to_json(pdf_path)

        # Save output
        with open("extracted_data.json", "w", encoding="utf-8") as f:
            json.dump(extracted_data, f, indent=4, ensure_ascii=False)

        print("\n📄 === Final JSON Output ===\n")
        print(json.dumps(extracted_data, indent=4, ensure_ascii=False))
        print("\n✅ Extracted data saved to extracted_data.json")

    except Exception as e:
        print(f"\n❌ Error: {e}")

    finally:
        if os.path.exists("downloaded.pdf"):
            os.remove("downloaded.pdf")


if __name__ == "__main__":
    main()


=== NLP-Ready PDF Extractor ===
📎 Enter Google Drive file link or file ID: https://drive.google.com/file/d/1zeC7WzV6dQUvbrKODZsMbBM0IEka2g7o/view?usp=drive_link 

📥 Downloading PDF from: https://drive.google.com/uc?id=1zeC7WzV6dQUvbrKODZsMbBM0IEka2g7o



Downloading...
From: https://drive.google.com/uc?id=1zeC7WzV6dQUvbrKODZsMbBM0IEka2g7o
To: /content/downloaded.pdf
100%|██████████| 229k/229k [00:00<00:00, 46.0MB/s]


📄 === Final JSON Output ===

{
    "Pages": [
        {
            "Page_Number": 1,
            "Headers": [
                "Name:",
                "PRICE",
                "QTY",
                "Discount:",
                "Taxes:",
                "Shipping:",
                "TOTAL:",
                "Payment Type:",
                "Track #:",
                "Arrival Date:",
                "Method:",
                "Company:",
                "Subtotal:",
                "PRODUCT NAME",
                "ITEM #",
                "SIZE",
                "COLOUR",
                "TOTAL",
                "Address:",
                "Email:",
                "Phone:",
                "DATE:",
                "PURCHASE ORDER #:",
                "CLIENT INFORMATION",
                "ORDER INFORMATION",
                "SHIPPING INFO",
                "PAYMENT INFO",
                "NOTES",
                "22ND SEPTEMBER, 2022",
                "FASHION QUEEN",
              

FINAL CODE


In [ ]:
#final code


import fitz  # PyMuPDF
import gdown
import os
import json


def get_file_id(drive_input):
    if "drive.google.com" in drive_input:
        try:
            return drive_input.split("/d/")[1].split("/")[0]
        except IndexError:
            raise ValueError("Invalid Google Drive link format.")
    return drive_input


def download_pdf(file_id, output_path="downloaded.pdf"):
    url = f"https://drive.google.com/uc?id={file_id}"
    print(f"Downloading PDF from: {url}")
    gdown.download(url, output_path, quiet=False)
    return output_path


def extract_text_lines_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    all_lines = []

    print("\nExtracting visible text from pages...\n")
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        lines = [line.strip() for line in text.splitlines() if line.strip()]
        all_lines.extend(lines)

    return all_lines


def extract_table_by_chunking(lines):
    """
    Locate product entries and extract chunks of 7 lines.
    """
    product_keywords = [
        "Poshmark black dress",
        "Waterproof French overcoat",
        "Signature perfume",
        'Classic blazer "mindsweeper"'
    ]

    product_indices = [i for i, line in enumerate(lines) if line in product_keywords]

    table_items = []
    used_indices = set()

    for start in product_indices:
        if any(i in used_indices for i in range(start, start + 7)):
            continue  # avoid duplicate entries

        row = lines[start:start + 7]
        if len(row) == 7:
            table_items.append({
                "Product Name": row[0],
                "Item #": row[1],
                "Price": row[2],
                "Qty": row[3],
                "Size": row[4],
                "Color": row[5],
                "Total": row[6]
            })
            used_indices.update(range(start, start + 7))

    # Everything not in used_indices = Header
    headers = [line for i, line in enumerate(lines) if i not in used_indices]

    return headers, table_items


def extract_structured_json(pdf_path):
    lines = extract_text_lines_from_pdf(pdf_path)
    headers, table = extract_table_by_chunking(lines)

    return {
        "Pages": [
            {
                "Page_Number": 1,
                "Headers": headers,
                "List_items": table
            }
        ]
    }


def main():
    print("=== Advanced PDF Extractor + Smart Table Detection ===")
    drive_input = input("Enter Google Drive file link or file ID: ").strip()

    try:
        file_id = get_file_id(drive_input)
        pdf_path = download_pdf(file_id)

        extracted_data = extract_structured_json(pdf_path)

        with open("extracted_data.json", "w", encoding="utf-8") as f:
            json.dump(extracted_data, f, indent=4, ensure_ascii=False)

        print("\n=== Final Structured JSON Output ===\n")
        print(json.dumps(extracted_data, indent=4, ensure_ascii=False))
        print("\n✅ Data saved to extracted_data.json")

    except Exception as e:
        print(f"\n❌ Error: {e}")
    finally:
        if os.path.exists("downloaded.pdf"):
            os.remove("downloaded.pdf")


if __name__ == "__main__":
    main()


=== Advanced PDF Extractor + Smart Table Detection ===
Enter Google Drive file link or file ID: https://drive.google.com/file/d/1zeC7WzV6dQUvbrKODZsMbBM0IEka2g7o/view?usp=drive_link 


Downloading...
From: https://drive.google.com/uc?id=1zeC7WzV6dQUvbrKODZsMbBM0IEka2g7o
To: /content/downloaded.pdf
100%|██████████| 229k/229k [00:00<00:00, 83.3MB/s]


Extracting visible text from pages...


=== Final Structured JSON Output ===

{
    "Pages": [
        {
            "Page_Number": 1,
            "Headers": [
                "Name:",
                "PRICE",
                "QTY",
                "Discount:",
                "Taxes:",
                "Shipping:",
                "TOTAL:",
                "Payment Type:",
                "Track #:",
                "Arrival Date:",
                "Method:",
                "Company:",
                "Subtotal:",
                "PRODUCT NAME",
                "ITEM #",
                "SIZE",
                "COLOUR",
                "TOTAL",
                "Address:",
                "Email:",
                "Phone:",
                "DATE:",
                "PURCHASE ORDER #:",
                "CLIENT INFORMATION",
                "ORDER INFORMATION",
                "SHIPPING INFO",
                "PAYMENT INFO",
                "NOTES",
                "22ND SEPTEMBER, 2022"

In [ ]:
import fitz  # PyMuPDF
import gdown
import os
import json
import re


def get_file_id(drive_input):
    if "drive.google.com" in drive_input:
        try:
            return drive_input.split("/d/")[1].split("/")[0]
        except IndexError:
            raise ValueError("Invalid Google Drive link format.")
    return drive_input


def download_pdf(file_id, output_path="downloaded.pdf"):
    url = f"https://drive.google.com/uc?id={file_id}"
    print(f"Downloading PDF from: {url}")
    gdown.download(url, output_path, quiet=False)
    return output_path


def extract_text_lines_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    all_lines = []

    print("\nExtracting visible text from pages...\n")
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        lines = [line.strip() for line in text.splitlines() if line.strip()]
        all_lines.extend(lines)

    return all_lines


def detect_and_extract_table(lines):
    """
    Generalized table extractor.
    Looks for numbered item patterns like '1)', '2)', etc. and extracts following lines as rows.
    """
    headers = []
    table_items = []
    item_pattern = re.compile(r"^\d+\)\s*")

    i = 0
    while i < len(lines):
        if item_pattern.match(lines[i]):
            row = lines[i:i + 6]  # try 6-line chunk
            if len(row) == 6:
                table_items.append({
                    "Item No.": row[0],
                    "Details": row[1],
                    "Unit": row[2],
                    "Quantity": row[3],
                    "Unit Price": row[4],
                    "Total": row[5]
                })
                i += 6
            else:
                break
        else:
            headers.append(lines[i])
            i += 1

    return headers, table_items


def extract_structured_json(pdf_path):
    lines = extract_text_lines_from_pdf(pdf_path)
    headers, table = detect_and_extract_table(lines)

    return {
        "Pages": [
            {
                "Page_Number": 1,
                "Headers": headers,
                "List_items": table
            }
        ]
    }


def main():
    print("=== Robust PDF Extractor for Tables & Headers ===")
    drive_input = input("Enter Google Drive file link or file ID: ").strip()

    try:
        file_id = get_file_id(drive_input)
        pdf_path = download_pdf(file_id)

        extracted_data = extract_structured_json(pdf_path)

        with open("extracted_data.json", "w", encoding="utf-8") as f:
            json.dump(extracted_data, f, indent=4, ensure_ascii=False)

        print("\n=== Final Structured JSON Output ===\n")
        print(json.dumps(extracted_data, indent=4, ensure_ascii=False))
        print("\n✅ Data saved to extracted_data.json")

    except Exception as e:
        print(f"\n❌ Error: {e}")
    finally:
        if os.path.exists("downloaded.pdf"):
            os.remove("downloaded.pdf")


if __name__ == "__main__":
    main()


=== Robust PDF Extractor for Tables & Headers ===
Enter Google Drive file link or file ID: https://drive.google.com/file/d/1ozEKcPo6fWJp1rkiHy5Jvv7RFmn3VCMw/view?usp=drive_link


Downloading...
From: https://drive.google.com/uc?id=1ozEKcPo6fWJp1rkiHy5Jvv7RFmn3VCMw
To: /content/downloaded.pdf
100%|██████████| 1.27M/1.27M [00:00<00:00, 135MB/s]


Extracting visible text from pages...


=== Final Structured JSON Output ===

{
    "Pages": [
        {
            "Page_Number": 1,
            "Headers": [
                "PURCHASE ORDER",
                "DATE:",
                "PURCHASE ORDER NO.:",
                "VENDOR INFORMATION",
                "CUSTOMER INFORMATION",
                "Item No.",
                "Details",
                "Unit",
                "Quantity",
                "Unit Price",
                "Total",
                "Taxes (%)",
                "Discount (%)",
                "Additional Notes:",
                "ADDRESS",
                "CONTACT NO.",
                "EMAIL ADDRESS",
                "CONTACT NO.",
                "EMAIL ADDRESS",
                "ADDRESS",
                "VENDOR NAME",
                "SALES PERSON",
                "CUSTOMER NAME",
                "CONTACT PERSON",
                "Authorized By",
                "Subtotal",
                "Total",
     

In [ ]:
import fitz  # PyMuPDF
import gdown
import os
import json
import re


def get_file_id(drive_input):
    if "drive.google.com" in drive_input:
        try:
            return drive_input.split("/d/")[1].split("/")[0]
        except IndexError:
            raise ValueError("Invalid Google Drive link format.")
    return drive_input


def download_pdf(file_id, output_path="downloaded.pdf"):
    url = f"https://drive.google.com/uc?id={file_id}"
    print(f"Downloading PDF from: {url}")
    gdown.download(url, output_path, quiet=False)
    return output_path


def extract_text_lines_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    all_lines = []

    print("\nExtracting visible text from pages...\n")
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        lines = [line.strip() for line in text.splitlines() if line.strip()]
        all_lines.extend(lines)

    return all_lines


def detect_and_extract_table(lines):
    """
    Generalized table extractor.
    Looks for numbered item patterns like '1)', '2)', etc. and extracts following lines as rows.
    """
    headers = []
    table_items = []
    item_pattern = re.compile(r"^\d+\)\s*")

    i = 0
    while i < len(lines):
        if item_pattern.match(lines[i]):
            row = lines[i:i + 6]  # try 6-line chunk
            if len(row) == 6:
                table_items.append({
                    "Item No.": row[0],
                    "Details": row[1],
                    "Unit": row[2],
                    "Quantity": row[3],
                    "Unit Price": row[4],
                    "Total": row[5]
                })
                i += 6
            else:
                break
        else:
            headers.append(lines[i])
            i += 1

    return headers, table_items


def extract_structured_json(pdf_path):
    lines = extract_text_lines_from_pdf(pdf_path)
    headers, table = detect_and_extract_table(lines)

    return {
        "Pages": [
            {
                "Page_Number": 1,
                "Headers": headers,
                "List_items": table
            }
        ]
    }


def main():
    print("=== Robust PDF Extractor for Tables & Headers ===")
    drive_input = input("Enter Google Drive file link or file ID: ").strip()

    try:
        file_id = get_file_id(drive_input)
        pdf_path = download_pdf(file_id)

        extracted_data = extract_structured_json(pdf_path)

        with open("extracted_data.json", "w", encoding="utf-8") as f:
            json.dump(extracted_data, f, indent=4, ensure_ascii=False)

        print("\n=== Final Structured JSON Output ===\n")
        print(json.dumps(extracted_data, indent=4, ensure_ascii=False))
        print("\n✅ Data saved to extracted_data.json")

    except Exception as e:
        print(f"\n❌ Error: {e}")
    finally:
        if os.path.exists("downloaded.pdf"):
            os.remove("downloaded.pdf")


if __name__ == "__main__":
    main()


=== Robust PDF Extractor for Tables & Headers ===
Enter Google Drive file link or file ID: https://drive.google.com/file/d/1LUUJUPgXMKHA6aPojpI7rFavGUdhputo/view?usp=drive_link


Downloading...
From: https://drive.google.com/uc?id=1LUUJUPgXMKHA6aPojpI7rFavGUdhputo
To: /content/downloaded.pdf
100%|██████████| 160k/160k [00:00<00:00, 75.8MB/s]


Extracting visible text from pages...


=== Final Structured JSON Output ===

{
    "Pages": [
        {
            "Page_Number": 1,
            "Headers": [
                "Purchase Order To:",
                "#",
                "Service Description",
                "#",
                "Item Description",
                "Comments",
                "Date:",
                "Approved By:",
                "TOTAL",
                "TOTAL MATERIALS:",
                "Accepted By:",
                "TOTAL SERVICES:",
                "Ship To:",
                "PURCHASE ORDER",
                "Total Job Cost",
                "TAX",
                "Qty",
                "Price",
                "TOTAL PRICE",
                "Time range",
                "[Days]",
                "Price",
                "[per hour]",
                "TOTAL PRICE",
                "PO #:",
                "Date:",
                "0001253",
                "01/01/2022",
                "270 Con

In [ ]:
import fitz  # PyMuPDF
import gdown
import os
import json
import re


def get_file_id(drive_input):
    if "drive.google.com" in drive_input:
        try:
            return drive_input.split("/d/")[1].split("/")[0]
        except IndexError:
            raise ValueError("Invalid Google Drive link format.")
    return drive_input


def download_pdf(file_id, output_path="downloaded.pdf"):
    url = f"https://drive.google.com/uc?id={file_id}"
    print(f"Downloading PDF from: {url}")
    gdown.download(url, output_path, quiet=False)
    return output_path


def extract_text_lines_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    all_lines = []

    print("\nExtracting visible text from pages...\n")
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        lines = [line.strip() for line in text.splitlines() if line.strip()]
        all_lines.extend(lines)

    return all_lines


def is_price(value):
    return bool(re.match(r"^\$?\d{1,3}(,\d{3})*(\.\d{2})?\$?$", value.replace('$', '').replace(',', '')))


def group_table_rows(lines, section_keyword):
    """
    Extracts rows below a section keyword and clusters them into tables.
    """
    table = []
    collecting = False
    current_row = []

    for line in lines:
        if line.upper() == section_keyword:
            collecting = True
            continue
        if collecting:
            # Stop if we hit another section or irrelevant text
            if re.match(r"^[A-Z\s]{5,}$", line) and not any(char.isdigit() for char in line):
                break

            # Heuristic: lines starting with numbers or containing consistent pattern
            if re.match(r"^\d+\)?$", line) or is_price(line) or re.match(r"^\d{1,4}(\.\d{2})?\$?$", line):
                if current_row:
                    table.append(current_row)
                    current_row = []
                current_row.append(line)
            else:
                current_row.append(line)

    if current_row:
        table.append(current_row)

    return table


def clean_and_format_table(table_rows):
    """
    Converts clustered table rows into structured dictionaries with guessed column names.
    """
    formatted = []

    for row in table_rows:
        # Remove numbering if present (like "1)", "2)")
        if re.match(r"^\d+\)?$", row[0]):
            row = row[1:]

        if len(row) == 5:
            formatted.append({
                "Description": row[0],
                "Quantity": row[1],
                "Unit Price": row[2],
                "Total": row[3],
                "Comments": row[4]
            })
        elif len(row) == 4:
            formatted.append({
                "Description": row[0],
                "Quantity": row[1],
                "Unit Price": row[2],
                "Total": row[3]
            })
        elif len(row) == 3:
            formatted.append({
                "Description": row[0],
                "Unit": row[1],
                "Price": row[2]
            })
        else:
            formatted.append({"Row": row})  # fallback

    return formatted


def extract_structured_json(pdf_path):
    lines = extract_text_lines_from_pdf(pdf_path)

    # Define likely section headers to find tables under
    section_keywords = ["SERVICES", "MATERIALS", "PRODUCTS", "ITEMS"]

    all_tables = []
    headers = []
    collected_sections = set()

    i = 0
    while i < len(lines):
        line = lines[i]
        upper_line = line.upper()

        if upper_line in section_keywords and upper_line not in collected_sections:
            table_rows = group_table_rows(lines[i + 1:], upper_line)
            structured_rows = clean_and_format_table(table_rows)
            if structured_rows:
                all_tables.extend(structured_rows)
            collected_sections.add(upper_line)
            i += len(table_rows) + 1
        else:
            headers.append(line)
            i += 1

    return {
        "Pages": [
            {
                "Page_Number": 1,
                "Headers": headers,
                "List_items": all_tables
            }
        ]
    }


def main():
    print("=== Universal NLP-Ready PDF Table Extractor ===")
    drive_input = input("Enter Google Drive file link or file ID: ").strip()

    try:
        file_id = get_file_id(drive_input)
        pdf_path = download_pdf(file_id)

        extracted_data = extract_structured_json(pdf_path)

        with open("extracted_data.json", "w", encoding="utf-8") as f:
            json.dump(extracted_data, f, indent=4, ensure_ascii=False)

        print("\n=== Final Structured JSON Output ===\n")
        print(json.dumps(extracted_data, indent=4, ensure_ascii=False))
        print("\n✅ Data saved to extracted_data.json")

    except Exception as e:
        print(f"\n❌ Error: {e}")
    finally:
        if os.path.exists("downloaded.pdf"):
            os.remove("downloaded.pdf")


if __name__ == "__main__":
    main()


=== Universal NLP-Ready PDF Table Extractor ===
Enter Google Drive file link or file ID: https://drive.google.com/file/d/1zeC7WzV6dQUvbrKODZsMbBM0IEka2g7o/view?usp=drive_link 


Downloading...
From: https://drive.google.com/uc?id=1zeC7WzV6dQUvbrKODZsMbBM0IEka2g7o
To: /content/downloaded.pdf
100%|██████████| 229k/229k [00:00<00:00, 64.3MB/s]


Extracting visible text from pages...


=== Final Structured JSON Output ===

{
    "Pages": [
        {
            "Page_Number": 1,
            "Headers": [
                "Name:",
                "PRICE",
                "QTY",
                "Discount:",
                "Taxes:",
                "Shipping:",
                "TOTAL:",
                "Payment Type:",
                "Track #:",
                "Arrival Date:",
                "Method:",
                "Company:",
                "Subtotal:",
                "PRODUCT NAME",
                "ITEM #",
                "SIZE",
                "COLOUR",
                "TOTAL",
                "Address:",
                "Email:",
                "Phone:",
                "DATE:",
                "PURCHASE ORDER #:",
                "CLIENT INFORMATION",
                "ORDER INFORMATION",
                "SHIPPING INFO",
                "PAYMENT INFO",
                "NOTES",
                "22ND SEPTEMBER, 2022"

In [ ]:
import fitz  # PyMuPDF
import gdown
import os
import json
import re


def get_file_id(drive_input):
    if "drive.google.com" in drive_input:
        try:
            return drive_input.split("/d/")[1].split("/")[0]
        except IndexError:
            raise ValueError("Invalid Google Drive link format.")
    return drive_input


def download_pdf(file_id, output_path="downloaded.pdf"):
    url = f"https://drive.google.com/uc?id={file_id}"
    print(f"Downloading PDF from: {url}")
    gdown.download(url, output_path, quiet=False)
    return output_path


def extract_text_lines_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    all_lines = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
        all_lines.extend(lines)
    return all_lines


def detect_section_tables(lines, used_indices):
    section_keywords = ["SERVICES", "MATERIALS"]
    tables = []

    for i, line in enumerate(lines):
        if line.upper() in section_keywords:
            # after header, take rows until blank or new section
            j = i+1
            while j < len(lines) and lines[j].strip():
                row = []
                # group until next numeric marker or blank
                while j < len(lines) and lines[j].strip():
                    row.append(lines[j]); j += 1
                if len(row) >= 3:
                    tables.append(row)
                # skip possible blanks
                while j < len(lines) and not lines[j].strip(): j+=1
    # mark all indices in tables
    for row in tables:
        start = None
        for idx, val in enumerate(lines):
            if val == row[0] and start is None:
                start = idx
        if start is not None:
            for k in range(len(row)):
                used_indices.add(start+k)
    return tables


def detect_stacked_blocks(lines, used_indices):
    # dynamic detection of block size 4..8
    best_size, best_count = 0, 0
    for size in range(4, 9):
        count = 0
        for i in range(len(lines)-size):
            if all((i+j) not in used_indices for j in range(size)):
                block = lines[i:i+size]
                # heuristic: second line numeric, last line price-like
                if re.match(r"^\d+", block[1]) and re.search(r"\d", block[-1]):
                    count += 1
        if count > best_count:
            best_count, best_size = count, size

    products = []
    i = 0
    while i < len(lines)-best_size and best_size>0:
        if i in used_indices:
            i += 1; continue
        block = lines[i:i+best_size]
        if re.match(r"^\d+", block[1]) and re.search(r"\d", block[-1]):
            # generic mapping: name=block[0], rest index-based
            item = {"Name": block[0]}
            for idx in range(1, best_size):
                item[f"Field_{idx}"] = block[idx]
            products.append(item)
            for k in range(best_size): used_indices.add(i+k)
            i += best_size
        else:
            i += 1
    return products


def extract_structured_json(pdf_path):
    lines = extract_text_lines_from_pdf(pdf_path)
    used = set()

    # 1. sections
    sec_tables = detect_section_tables(lines, used)
    # 2. stacked
    stacked = detect_stacked_blocks(lines, used)

    headers = [ln for idx, ln in enumerate(lines) if idx not in used]
    list_items = []
    # format section tables into dicts
    for row in sec_tables:
        # assume fixed columns per pdf-specific sections
        list_items.append({f"Col_{i+1}": v for i, v in enumerate(row)})
    list_items.extend(stacked)

    return {"Pages": [{"Page_Number":1, "Headers":headers, "List_items": list_items}]}


def main():
    print("=== Universal PDF Extractor Master ===")
    link = input("Enter Drive link or ID: ").strip()
    file_id = get_file_id(link)
    pdf = download_pdf(file_id)
    data = extract_structured_json(pdf)
    with open("extracted_data.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    print(json.dumps(data, indent=4, ensure_ascii=False))
    if os.path.exists(pdf): os.remove(pdf)

if __name__ == "__main__": main()


=== Universal PDF Extractor Master ===
Enter Drive link or ID: https://drive.google.com/file/d/1LUUJUPgXMKHA6aPojpI7rFavGUdhputo/view?usp=drive_link


Downloading...
From: https://drive.google.com/uc?id=1LUUJUPgXMKHA6aPojpI7rFavGUdhputo
To: /content/downloaded.pdf
100%|██████████| 160k/160k [00:00<00:00, 72.8MB/s]

{
    "Pages": [
        {
            "Page_Number": 1,
            "Headers": [
                "Purchase Order To:",
                "#",
                "Service Description",
                "#",
                "Item Description",
                "Comments",
                "Date:",
                "Approved By:",
                "TOTAL",
                "TOTAL MATERIALS:",
                "Accepted By:",
                "TOTAL SERVICES:",
                "Ship To:",
                "PURCHASE ORDER",
                "Total Job Cost",
                "TAX",
                "Qty",
                "Price",
                "TOTAL PRICE",
                "Time range",
                "[Days]",
                "Price",
                "[per hour]",
                "TOTAL PRICE",
                "PO #:",
                "www.constructionmasters123.com",
                "Company Name",
                "Street Address",
                "City, State ZIP",
                "Phone Number",
  